In [31]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from matplotlib import pyplot as plt
import cv2
import xml.etree.ElementTree as ET 
import numpy as np
import sys
from getBoundingBoxes import *
from corner_detector import *
from getBoxFeatures import *
from anms import *
from utils import *
from estimateAllTranslation import *
from estimateFeatureTranslation import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
cap = cv2.VideoCapture('vids/Easy.mp4')
ret,firstFrame = cap.read()
boxes = getBoundingBoxes()
gray = cv2.cvtColor(firstFrame,cv2.COLOR_BGR2GRAY)

boxesData = []

for box in boxes:
    boximg = gray[box[1]:box[3],box[0]:box[2]]
    x,y = getBoxFeatures(boximg,box)
    boxData = {
        'coords': box,
        '_x': x,
        '_y': y
    }
    boxesData.append(boxData)

currentFrame = firstFrame
_,nextFrame = cap.read()


In [40]:
# Imag, Ix, Iy,Iori=findDerivatives(gray)
# plt.imshow(Imag)
_x = boxesData[0]['_x']
_y = boxesData[0]['_y']
X,Y=estimateAllTranslation(_x,_y,currentFrame,nextFrame)

(100,)
